In [ ]:
import numpy as np
import tensorflow as tf
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
keras = tf.keras ## instantiate keras

In [ ]:
base_path = os.path.join("/content/drive/MyDrive") ##base path for the datasets (in colab

In [ ]:
train_data = np.load(os.path.join(base_path, "traindata.npy")) ##loading data form presaved pre-processed image
train_label = np.load(os.path.join(base_path, "trainlabel.npy"))
test_data = np.load(os.path.join(base_path, "testdata.npy"))
test_label = np.load(os.path.join(base_path, "testlabel.npy"))

In [ ]:
train_label = keras.utils.to_categorical(train_label) ##change label to categorical to fit the model
test_label = keras.utils.to_categorical(test_label)

In [ ]:
from sklearn.model_selection import train_test_split
train_data, validation_data, train_label, validation_label = train_test_split(train_data, train_label, test_size = 0.1) ##create validation split

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_label))
validation_dataset = tf.data.Dataset.from_tensor_slices((validation_data, validation_label)) ##create tensor dataset

In [ ]:
batch_size = 64
epochs = 10
shuffle_buffer = 1000
train_dataset = train_dataset.shuffle(shuffle_buffer).batch(batch_size)
validation_dataset = validation_dataset.shuffle(shuffle_buffer).batch(batch_size) ##shuffle, and batch them datasets

In [ ]:
layers = tf.keras.layers
input_shape = (256,256,3)

In [ ]:
input_layer = layers.Conv2D(filters=64, kernel_size=(5,5), strides=(4,4), padding="valid", activation='relu', input_shape=input_shape)
input_pooling = layers.MaxPool2D(pool_size=(5,5), strides=(2,2))
hidden_layer1 = layers.Conv2D(filters = 128, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same")
hidden_layer1_pooling = layers.MaxPool2D(pool_size = (5,5), strides =(2,2))
hidden_layer2 = layers.Conv2D(filters = 256, kernel_size = (3,3), strides=(1,1), activation='relu', padding="same")
hidden_layer2_pooling = layers.MaxPool2D(pool_size= (3,3), strides=(1,1))
hidden_layer3 = layers.Conv2D(filters = 256, kernel_size = (3,3), strides = (1,1), activation ='relu', padding="valid")
hidden_layer3_pooling = layers.MaxPool2D(pool_size=(3,3), strides=(2,2))
flatten_layer = layers.Flatten()
dense_layer = layers.Dense(4096, activation = 'relu')
dropout_layer1 = layers.Dropout(0.3)
output_layer = layers.Dense(2, activation = 'softmax')

In [ ]:
model = keras.Sequential([
    input_layer,
    layers.BatchNormalization(),
    input_pooling,
    hidden_layer1,
    layers.BatchNormalization(),
    hidden_layer1_pooling,
    hidden_layer2,
    layers.BatchNormalization(),
    hidden_layer2_pooling,
    hidden_layer3,
    layers.BatchNormalization(),
    hidden_layer3_pooling,
    flatten_layer,
    dense_layer,
    dropout_layer1,
    output_layer
])

In [ ]:
model.compile(optimizer=tf.optimizers.SGD(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
import time
root_logdir = os.path.join(os.curdir, "logs\\fit\\")
def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)
run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [ ]:
model.fit(train_dataset, epochs=epochs, validation_data=validation_dataset, callbacks=[tensorboard_cb])

Epoch 1/10
67/67 [==============================] - 171s 2s/step - loss: 0.3170 - accuracy: 0.8809 - val_loss: 0.5833 - val_accuracy: 0.7290
Epoch 2/10
67/67 [==============================] - 142s 2s/step - loss: 0.1681 - accuracy: 0.9351 - val_loss: 0.6484 - val_accuracy: 0.7290
Epoch 3/10
67/67 [==============================] - 137s 2s/step - loss: 0.1179 - accuracy: 0.9568 - val_loss: 0.8295 - val_accuracy: 0.7290
Epoch 4/10
67/67 [==============================] - 136s 2s/step - loss: 0.0981 - accuracy: 0.9601 - val_loss: 1.2546 - val_accuracy: 0.7290
Epoch 5/10
67/67 [==============================] - 138s 2s/step - loss: 0.0782 - accuracy: 0.9696 - val_loss: 1.4279 - val_accuracy: 0.7290
Epoch 6/10
67/67 [==============================] - 142s 2s/step - loss: 0.0653 - accuracy: 0.9764 - val_loss: 1.8185 - val_accuracy: 0.7290
Epoch 7/10
67/67 [==============================] - 140s 2s/step - loss: 0.0577 - accuracy: 0.9780 - val_loss: 1.1534 - val_accuracy: 0.7290
Epoch 8/10
67

In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_data, test_label))
test_batches = test_dataset.batch(batch_size)

In [ ]:
predict = model.predict(test_batches)

10/10 [==============================] - 8s 623ms/step


In [ ]:
model.save("Self_constructed_pneumonia_prediction.h5")